In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
import scipy.stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### Read ZCruit-like data in
Load in the cleaned (but not filled-in) Zcruit data, `cleaned_zcruit_data.csv`. Show the first 5 rows. Data saved as `data` to be used throughout the notebook.

In [14]:
data = pd.read_csv("cleaned_zcruit_data.csv")
data.head(5)

C:\Users\AlvinYu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,17,19,21,23,25,27,29,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Full Name,Last Name,Grad Year,Position,Height,Height Source,Weight,Weight Source,Zcruit Rating,Committed To,...,Hand Size Source,Powerball Toss,Powerball Toss Source,SPARQ Rating,SPARQ Rating Source,GPA,ACT,SAT,Unnamed: 37,Unnamed: 38
0,Korey Foreman,Foreman,2021,DE,"6' 4""",nike_opening,248.0,nike_opening,96.6970,USC,...,NaN,44.5,nike_opening,98.76,nike_opening,3.03,NaN,NaN,NaN,NaN
1,J.T. Tuimoloau,Tuimoloau,2021,DE,"6' 4""",national_preps,275.0,national_preps,96.6970,NaN,...,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN
2,JC Latham,Latham,2021,OT,"6' 6""",national_preps,279.0,nike_opening,96.6970,Alabama,...,NaN,NaN,NaN,NaN,NaN,2.35,NaN,NaN,NaN,NaN
3,Emeka Egbuka,Egbuka,2021,WR,"6' 0.5""",nike_opening,190.0,nike_opening,96.3939,Ohio State,...,national_combine,41.0,nike_opening,NaN,NaN,4.00,NaN,NaN,NaN,NaN
4,Dallas Turner,Turner,2021,DE,"6' 2.5""",rivals_camp,223.0,rivals_camp,96.3939,Alabama,...,rivals_camp,NaN,NaN,NaN,NaN,3.70,NaN,NaN,NaN,NaN


### Define function to parse heights. 

In [15]:
def height_str_to_val(height_str):
    """Takes in a height string like "6' 4"" and converts string into corresponding float in inches such as 76.0 
        for stated example."""
    match = re.match("^(\d)'\s*(\d*\.*\d*)", height_str)
    groups = match.groups()
    feet, inches = int(groups[0]), float(groups[1])
    return feet * 12 + inches 

### Apply height function to data.
Only run this cell once! If you want to rerun this cell, you'll have to reload in the `data` cell. To verify the `height_str_to_val` function worked, check that the values in the height column are expressed as floats (76.0, 77.0, etc.) instead of 6' 4".

In [16]:
data['Height'] = data['Height'].apply(height_str_to_val)
data.head(5)

,Full Name,Last Name,Grad Year,Position,Height,Height Source,Weight,Weight Source,Zcruit Rating,Committed To,...,Hand Size Source,Powerball Toss,Powerball Toss Source,SPARQ Rating,SPARQ Rating Source,GPA,ACT,SAT,Unnamed: 37,Unnamed: 38
0,Korey Foreman,Foreman,2021,DE,76.0,nike_opening,248.0,nike_opening,96.6970,USC,...,NaN,44.5,nike_opening,98.76,nike_opening,3.03,NaN,NaN,NaN,NaN
1,J.T. Tuimoloau,Tuimoloau,2021,DE,76.0,national_preps,275.0,national_preps,96.6970,NaN,...,NaN,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN
2,JC Latham,Latham,2021,OT,78.0,national_preps,279.0,nike_opening,96.6970,Alabama,...,NaN,NaN,NaN,NaN,NaN,2.35,NaN,NaN,NaN,NaN
3,Emeka Egbuka,Egbuka,2021,WR,72.5,nike_opening,190.0,nike_opening,96.3939,Ohio State,...,national_combine,41.0,nike_opening,NaN,NaN,4.00,NaN,NaN,NaN,NaN
4,Dallas Turner,Turner,2021,DE,74.5,rivals_camp,223.0,rivals_camp,96.3939,Alabama,...,rivals_camp,NaN,NaN,NaN,NaN,3.70,NaN,NaN,NaN,NaN


### Define master function to fill in empty values called master_fillna. 
Define the `master_fillna` function. Before that, we define needed sets of metrics, a list of all positions, and 5 helper/supporting functions: `create_percentile`, `value_to_percentile`, `percentile_to_value`, `add_score`, and `nan_counter`. Descriptions of each function are located in the docstring of the function in the code. 

In [17]:
combine_speed_events = ["Forty Yard Dash", "Shuttle", "3 Cone"]
track_events = ["100 Meter", "200 Meter", "110 Meter Hurdles", "300 Meter Hurdles"]
field_events = ['High Jump', 'Long Jump', 'Triple Jump', 'Shotput', 'Discus']
combine_metrics = ["Height", "Weight", "Forty Yard Dash", "3 Cone", "Shuttle", "Vertical Jump", "Broad Jump"]
zcruit_metrics = ["Wingspan","Arm Length","Hand Size", "Powerball Toss"]
positions = ["WR", "RB", "LB", "QB", "DE", "CB", "OL", "DT", "S", "OT", "OG", 
             "TE", "ATH", "DB", "DL", "OC", "K", "FB", "LS", "P"]

def create_percentile(position, event, data):
    """Take in a football position: CB, WR, QB, etc. and an event/measurable: Forty Yard Dash, Weight, 3 Cone, etc.
    
        Returns a list of percentile values in intervals of 5 (0th percentile, 5th percentile, 10th percentile, etc.). 
        Function also reverse values for all speed events (since lower is better) such as 40, 3 Cone, and Shuttle.
        
        For example:
        Inputs: position = 'WR', event = 'Forty Yard Dash', data = data
        Outputs: [24.0,
                 5.45,
                 5.2737,
                 5.18,
                 5.101,
                 5.03625,
                 4.988,
                 4.92195,
                 4.896800000000001,
                 4.839,
                 4.8,
                 4.764,
                 4.7,
                 4.7,
                 4.68,
                 4.63,
                 4.6,
                 4.6,
                 4.54,
                 4.5,
                 4.1] where each value represents a perentile in an interval of 5 (0th percentile, 5th percentile, 10th percentile, etc.)."""
    sorted_athletes = data[data['Position'] == position]     
    event_values = (sorted_athletes[event].astype(float)).dropna()
    percentile = []
    for i in np.arange(0, 1.01, 0.05):
        percentile.append(event_values.quantile(i))
    if event in combine_speed_events or event in track_events:
        return percentile[::-1]
    return percentile

def value_to_percentile(event, value, metric_dict):
    """Take in an event/measurable, a player's value for said event/measurable, and dictionary where the 
        keys are events and the values are percentile lists returned from the create_percentile function.
        
        Returns the corresponding percentile for the specific value.
        
        For example: 
        Inputs: event = 'Weight', value = 200, 
        metric_dict = {'Weight': [150, 165, 170, 175, 180, 185, 190, 195, 200, 220, 230]} 
        
        Outputs: 80 (since 200 is the 80th percentile from as can be seen from the list of percentiles)
        """
    if event in combine_speed_events or event in track_events:
        return round((100 - scipy.stats.percentileofscore(metric_dict[event], value, kind='rank')), 2)
    else:
        return round((scipy.stats.percentileofscore(metric_dict[event], value, kind='rank')), 2)
    
def percentile_to_value(event, percentile, metric_dict):
    """Take in an event/measurable, a percentile, and dictionary where the 
        keys are events and the values are percentile lists returned from the create_percentile function.
        
        Returns the corresponding value for the specific percentile. Is the reverse of the value_to_percentile functionality.
        
        For example: 
        Inputs: event = 'Weight', percentile = 90, 
        metric_dict = {'Weight': [150, 165, 170, 175, 180, 185, 190, 195, 200, 220, 230]} 
        
        Outputs: 200 (since the 80th percentile for Weight is 200 as can be seen from the list of percentiles)
        """
    if event in combine_speed_events or event in track_events:
        return scipy.stats.scoreatpercentile(metric_dict[event], 100 - percentile)
    else:
        return scipy.stats.scoreatpercentile(metric_dict[event], percentile)
        
def add_score(metric, data, metric_dict):
    """Take in a metric (another name for event/measurable), a ZCruit-like datset, and dictionary where the 
        keys are events and the values are percentile lists returned from the create_percentile function.
        
        Returns a dataset with an additional column called "[name of metric] Percentile"
        with the values being the percentiles for the values in the metric column.
        
        For example:
        Inputs: event = 'Weight', data = data, 
        metric_dict = {'Weight': [150, 165, 170, 175, 180, 185, 190, 195, 200, 220, 230]} 
        
        Outputs: a dataframe with an additional column called "Weight Percentile" where the values
        are the percentiles corresponding to the weight values for each player in the dataset. 
        """
    data[metric + " " + "Percentile"] = data.apply(lambda x: value_to_percentile(metric, x[metric], metric_dict), axis = 1)
    return data

def nan_counter(data, metrics):
    """Take in a ZCruit-like datset and a list of metrics.
        
        Prints a statement that describes the number of missing values (NaNs) for that metric. 
        
        Returns a dictionary where the keys are metrics and 
        the values are the percentage of players with missing values for each metric.
        
        For example:
        Inputs: data = data, metrics = ['Forty Yard Dash', 'Weight']
         
        Printed: {'Forty Yard Dash': 'There are 35994 NaNs out of 71370 players', 'Weight': 'There are 0 NaNs out of 71370 players'}
        Outputs: {'Forty Yard Dash': 0.5043295502311895, 'Weight': 0.0}
        """
    nan_dict, readable_dict = {}, {}
    for metric in metrics:
        nan_dict[metric] = data[metric].isna().sum() / data.shape[0]
        readable_dict[metric] = "There are {} NaNs out of {} players".format(data[metric].isna().sum(), data.shape[0])
    print(readable_dict)
    return nan_dict

def master_fillna(data, position, metrics):
    """Take in a ZCruit-like datset, a football position, and a list of metrics.
        
        Returns a dataset that fills in all possible empty values for given metrics. We fill in values using two methods:
        1) Linear Regression Method (use one metric to predict another metric) if the metric has a strong correlation (> 0.4) with another metric in that position.
            Common Ex: Shuttle is used to predict/fill-in 3 Cone and vice-versa.
        2) Mean Percentile Method (used if a metric doesn't have a correlation > 0.4 with any other metrics).
            This method fills in empty values using the mean percentile of all other events the player has values for.
            
            Common Ex: Broad Jump doesn't have many correlated metrics. We decide to use the mean percentile method.
            For a player who is missing the Broad Jump, we consider the other metrics they have values for. 
            Say the player has values for [Forty Yard Dash, Weight, Height] with their percentiles being [80, 60, 40], 
            we would use the average of those 3 percentiles (60) to fill-in for the empty value for the Broad Jump.
            We then convert this percentile to a value using the percentile_to_value function to get an actual value. 
                    
        For example:
        Inputs: data = data, position = 'CB', metrics = ['Forty Yard Dash', 'Weight', 'Height', 'Broad Jump']
        
        Prints: 
        1) Position
        2) Empty Values Information from nan_counter function, 
        3) Metric we are filling in and Best Predictors (metrics with correlation > 0.4 with that metric) for that Metric,
        4, 5) The linear regression coefficient (slope) and intercept if we use the regression method and the RMSE of the linear regression model on the training data:
        a smaller dataset comprised of players who actually have the metric we want to fill-in.
        6, 7) Number of empty values for metric before our function. The number of empty values for metric after function (should be 0).
         
        Outputs: a dataframe with Full Name, Position, Forty Yard Dash, Weight, Height, and Broad Jump with no missing values 
        as long as there are some players with values for given metrics at that position. 
        
        If there are no players with a value for that metric, the metric will remain NaN.
        This happens very infrequently and for less value-rich positions. 
        For example, Powerball Toss for Kickers may be empty because there are no kickers with a Powerball Toss and we can't generate percentile lists with no data.
        """
    # Filtering data    
    print(position)
    data1 = data[data["Position"] == position]
    data_with_metrics = data1[metrics]
    
    # Getting NaN information
    nan_info = nan_counter(data_with_metrics, metrics)
    
    # Classifying metrics into complete, incomplete, and drop
    complete_metric, incomplete_metric, drop_metric = [], [], []
    for metric in nan_info.keys():
        if nan_info[metric] == 0:
            complete_metric.append(metric)
        elif metric in combine_metrics or metric in zcruit_metrics:
            incomplete_metric.append(metric)
        elif nan_info[metric] > 1:
            drop_metric.append(metric)
        else:
            incomplete_metric.append(metric)
    
    data_with_metrics = data_with_metrics.drop(columns = drop_metric)
    
    metric_dict = {}
    for metric in data_with_metrics.columns:
        metric_dict[metric] = create_percentile(position, metric, data)
    # We want to fill in the incomplete metrics
    completed_metrics = {}
    for metric in incomplete_metric:
        correlations = data_with_metrics.corr()[metric]
        correlations = correlations[correlations != 1].sort_values(ascending = False)
        metric_filter = correlations > 0.4 
        best_predictors = metric_filter[metric_filter].index.values

        print(metric, best_predictors)
    
    #IF there are no good predictors for linear regression (ie: no metrics with correlation > 0.4 with incomplete metric),
    #then we use the average approach. 
    
    #For T&F data, percentile method is a little tricky because some players will have NaNs for everything! No ht, wt guarantee like in ZCruit data. 
        if len(best_predictors) == 0:
            #For example, if we are predicting Hand Size with the average approach. We want the percentiles for Height, Weight, and Wingspan
            all_other_metrics = incomplete_metric + complete_metric
            all_other_metrics.remove(metric)
            
            all_other_percentiles = data_with_metrics.copy()
            for percentile_metrics in all_other_metrics:
                all_other_percentiles = add_score(percentile_metrics, all_other_percentiles, metric_dict)
                
            empty_metric = all_other_percentiles[all_other_percentiles[metric].isna()]
            full_metric = all_other_percentiles[all_other_percentiles[metric].notna()]
            
            metric_percentile_names =  [x + " " + "Percentile" for x in all_other_metrics]
            only_percentiles = empty_metric[metric_percentile_names]
            only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)

            only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
            empty_metric[metric] = only_percentiles['Predicted' + " " + metric]
            
            all_metric = empty_metric.append(full_metric).sort_index()
            
            completed_metrics[metric] = all_metric[metric].values
            #print(completed_metrics, len(completed_metrics[metric]), np.count_nonzero(~np.isnan(completed_metrics[metric])))

        else:
            full_metric = data_with_metrics[data_with_metrics[metric].notna()]
            empty_metric = data_with_metrics[data_with_metrics[metric].isna()]
            labels = [best_predictors[0]] + [metric]
            predictor = best_predictors[0]
            full_metric_with_no_predictor_nulls = full_metric[labels].dropna()
            X = full_metric_with_no_predictor_nulls[[predictor]]
            y = full_metric_with_no_predictor_nulls[metric]
            
            reg = LinearRegression(normalize = True).fit(X,y)
            metric_mean = np.median(y)
            
            empty_metric_with_no_predictor_nulls = empty_metric[[predictor]].dropna()
            if len(empty_metric_with_no_predictor_nulls) == 0:
                empty_metric_with_no_predictor_nulls[metric] = metric_mean
            else: 
                empty_metric_with_no_predictor_nulls[metric] = reg.predict(empty_metric_with_no_predictor_nulls)

            empty_metric_with_predictor_nulls = empty_metric[empty_metric[predictor].isna()]
            empty_metric_with_predictor_nulls[metric] = metric_mean
            
            empty_metric = empty_metric_with_no_predictor_nulls.append(empty_metric_with_predictor_nulls).sort_index()
            
            all_metric = empty_metric.append(full_metric).sort_index()
            
            completed_metrics[metric] = all_metric[metric].values
            
            train_y = reg.predict(X)
            train_error = mean_squared_error(y, train_y)
            print(reg.coef_, reg.intercept_)
            print("The RMSE of the model on the training data is {}. The average {} of the training data is {}. On average, predictions are {} from the actual values in the training data".format(np.sqrt(train_error), metric, np.mean(y), np.sqrt(train_error)/np.mean(y)))
            #print(completed_metrics, len(completed_metrics[metric]), np.count_nonzero(~np.isnan(completed_metrics[metric])))
    
    for metric in incomplete_metric:
        print(metric)
        print("There are {} NaN values for this metric".format(data1[metric].isna().sum()))
        data1[metric] = completed_metrics[metric]
        print("There are {} NaN values for this metric".format(data1[metric].isna().sum()))
    
    if 'Zcruit ID' in data.columns:
        all_metrics = ["Zcruit ID", "Full Name", "Position"] + complete_metric + incomplete_metric
        cleaned_dataset = data1[all_metrics]
        return cleaned_dataset
    
    else:
        all_metrics = ["Full Name", "Position"] + complete_metric + incomplete_metric
        cleaned_dataset = data1[all_metrics]
        return cleaned_dataset

### Option 1: Fill in Dataset Yourself

Code below iterates through all the positions and creates a dictionary called `cleaned_combine_metrics_position_dataframes` where the keys are the positions and the values are the cleaned datasets for each position using the combine and zcruit metrics. Has all the print statements associated with the `master_fillna` function which is why you see many outputs.

In [18]:
%%time
cleaned_combine_metrics_position_dataframes = {}
for position in positions:
    cleaned_combine_metrics_position_dataframes[position] = master_fillna(data, position, combine_metrics + zcruit_metrics)

WR
{'Height': 'There are 0 NaNs out of 12359 players', 'Weight': 'There are 0 NaNs out of 12359 players', 'Forty Yard Dash': 'There are 6235 NaNs out of 12359 players', '3 Cone': 'There are 9546 NaNs out of 12359 players', 'Shuttle': 'There are 8483 NaNs out of 12359 players', 'Vertical Jump': 'There are 8608 NaNs out of 12359 players', 'Broad Jump': 'There are 8801 NaNs out of 12359 players', 'Wingspan': 'There are 11947 NaNs out of 12359 players', 'Arm Length': 'There are 12082 NaNs out of 12359 players', 'Hand Size': 'There are 11963 NaNs out of 12359 players', 'Powerball Toss': 'There are 12340 NaNs out of 12359 players'}
Forty Yard Dash []


<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

3 Cone ['Powerball Toss' 'Shuttle']
[0.02733333] 5.962333333333332
The RMSE of the model on the training data is 0.0. The average 3 Cone of the training data is 7.0215. On average, predictions are 0.0 from the actual values in the training data
Shuttle ['3 Cone']
[0.38332496] 1.7086494219744428
The RMSE of the model on the training data is 0.24604069736686787. The average Shuttle of the training data is 4.6740147588193075. On average, predictions are 0.05264011991032302 from the actual values in the training data
Vertical Jump ['Powerball Toss']
[0.47905169] 14.928799067236692
The RMSE of the model on the training data is 2.2896714453272. The average Vertical Jump of the training data is 31.72222222222222. On average, predictions are 0.072178784616269 from the actual values in the training data
Broad Jump []


<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Wingspan ['Height' 'Arm Length' 'Weight']
[0.95391507] 6.044791501455151
The RMSE of the model on the training data is 2.3533220050715538. The average Wingspan of the training data is 73.49029126213593. On average, predictions are 0.03202221633164278 from the actual values in the training data
Arm Length ['Wingspan' 'Height' 'Weight']
[0.3974528] 2.384995730331898
The RMSE of the model on the training data is 1.4588901732158246. The average Arm Length of the training data is 31.645126353790616. On average, predictions are 0.04610157522853661 from the actual values in the training data
Hand Size []


...

<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_metric_with_predictor_nulls[metric] = metric_mean
<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_metric_with_predictor_nulls[metric] = metric_mean
<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

Wingspan []
Arm Length []
Hand Size []


<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Powerball Toss []
Forty Yard Dash
There are 98 NaN values for this metric
There are 0 NaN values for this metric
3 Cone
There are 160 NaN values for this metric
There are 0 NaN values for this metric
Shuttle
There are 140 NaN values for this metric
There are 0 NaN values for this metric
Vertical Jump
There are 150 NaN values for this metric
There are 0 NaN values for this metric
Broad Jump
There are 149 NaN values for this metric
There are 0 NaN values for this metric
Wingspan
There are 165 NaN values for this metric
There are 165 NaN values for this metric
Arm Length
There are 165 NaN values for this metric
There are 165 NaN values for this metric
Hand Size
There are 165 NaN values for this metric
There are 165 NaN values for this metric
Powerball Toss
There are 165 NaN values for this metric
There are 165 NaN values for this metric
Wall time: 56 s


<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:251: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

For example, here we access the filled-in dataset for RBs from `cleaned_combine_metrics_position_dataframes` and show the first 5 rows.

In [19]:
cleaned_combine_metrics_position_dataframes['RB'].head(5)

,Full Name,Position,Height,Weight,Forty Yard Dash,3 Cone,Shuttle,Vertical Jump,Broad Jump,Wingspan,Arm Length,Hand Size,Powerball Toss
23,Camar Wheaton,RB,71.0,190.0,4.50,7.750000,4.654,30.98400,109.968000,73.670213,31.000000,19.925888,37.17440
28,Donovan Edwards,RB,70.5,203.0,4.74,7.597472,4.500,33.10000,107.047600,73.182035,31.000000,9.731697,34.07616
41,Xavian Sorey,RB,74.0,207.0,4.60,7.750000,4.654,31.28608,110.476800,77.000000,32.787915,9.450000,37.39072
45,Will Shipley,RB,71.0,198.0,4.46,6.940000,4.210,36.30000,121.000000,73.000000,32.000000,9.450000,37.35248
48,TreVeyon Henderson,RB,69.5,177.5,4.30,7.750000,4.654,28.83292,105.982664,75.000000,32.094335,9.060000,34.00000


#### Combine all filled-in datasets by position into one master filled-in dataset called df. 
Master filled in-dataset for ZCruit data is saved as `df` variable. 

In [20]:
column_names = cleaned_combine_metrics_position_dataframes['DE'].columns
df = pd.DataFrame(columns = column_names)
for position in positions:
    df = df.append(cleaned_combine_metrics_position_dataframes[position])
df = df.sort_index()

Show first 5 rows of `df`.

In [21]:
df.head(5)

,Full Name,Position,Height,Weight,Forty Yard Dash,3 Cone,Shuttle,Vertical Jump,Broad Jump,Wingspan,Arm Length,Hand Size,Powerball Toss
0,Korey Foreman,DE,76.0,248.0,4.880,7.7580,4.7000,30.400,104.0000,79.3868,33.250000,9.691052,44.500000
1,J.T. Tuimoloau,DE,76.0,275.0,4.750,8.1930,4.9085,22.829,93.0000,79.3868,33.250000,8.753886,39.000000
2,JC Latham,OT,78.0,279.0,5.460,8.2187,4.8700,26.800,90.9524,79.5000,34.000000,10.220960,29.176265
3,Emeka Egbuka,WR,72.5,190.0,4.859,7.0830,4.2000,35.100,112.0000,75.0000,32.193956,9.000000,41.000000
4,Dallas Turner,DE,74.5,223.0,4.991,7.7520,4.4870,26.000,115.0000,83.5000,34.250000,9.840000,38.378617


Not every single event for every single position can be filled in. This occurs when there are 0 players with values for that metric at that position. This happens with infrequent metrics (Arm Length, Powerball Toss, Wingspan, Hand Size) at generally less popular positions (ATH, OL, OC, etc). 

Run the following code with any dataset (substitute name of dataset with `df`). The code below shows which metrics are still empty after running the cleaning code for each position. The information is saved in the `still_nan_events_for_positions_zcruit` dictionary.

In [22]:
still_nan_events_for_positions_zcruit = {}
for position in positions:
    series = df[df['Position'] == position].isna().sum() > 0
    still_nan_events_for_positions_zcruit[position] = series[series].index.values
still_nan_events_for_positions_zcruit

{'WR': array([], dtype=object),
 'RB': array([], dtype=object),
 'LB': array([], dtype=object),
 'QB': array([], dtype=object),
 'DE': array([], dtype=object),
 'CB': array([], dtype=object),
 'OL': array(['Arm Length', 'Powerball Toss'], dtype=object),
 'DT': array([], dtype=object),
 'S': array([], dtype=object),
 'OT': array([], dtype=object),
 'OG': array([], dtype=object),
 'TE': array([], dtype=object),
 'ATH': array(['Arm Length'], dtype=object),
 'DB': array(['Arm Length', 'Powerball Toss'], dtype=object),
 'DL': array(['Arm Length', 'Powerball Toss'], dtype=object),
 'OC': array([], dtype=object),
 'K': array(['Wingspan', 'Arm Length', 'Hand Size', 'Powerball Toss'],
       dtype=object),
 'FB': array(['Powerball Toss'], dtype=object),
 'LS': array(['Arm Length', 'Powerball Toss'], dtype=object),
 'P': array(['Wingspan', 'Arm Length', 'Hand Size', 'Powerball Toss'],
       dtype=object)}

### Option 2: Load in Cleaned Data. 

Do this if you don't want to run cells in option 1. Can just load in the already cleaned and filled-in ZCruit dataset called `cleaned_filled_zcruit_data.csv`. This dataset is the end result from all the work in option 1. Save this dataset as `cleaned`.

In [23]:
cleaned = pd.read_csv("cleaned_filled_zcruit_data.csv")

Show the first 5 rows of `cleaned`. Should look exactly like `df` from Option 1.

In [24]:
cleaned = cleaned.drop(columns = 'Unnamed: 0')
cleaned.head(5)

,Full Name,Position,Height,Weight,Forty Yard Dash,3 Cone,Shuttle,Vertical Jump,Broad Jump,Wingspan,Arm Length,Hand Size,Powerball Toss
0,Korey Foreman,DE,76.0,248.0,4.880,7.7580,4.7000,30.400,104.0000,79.3868,33.250000,9.691052,44.500000
1,J.T. Tuimoloau,DE,76.0,275.0,4.750,8.1930,4.9085,22.829,93.0000,79.3868,33.250000,8.753886,39.000000
2,JC Latham,OT,78.0,279.0,5.460,8.2187,4.8700,26.800,90.9524,79.5000,34.000000,10.220960,29.176265
3,Emeka Egbuka,WR,72.5,190.0,4.859,7.0830,4.2000,35.100,112.0000,75.0000,32.193956,9.000000,41.000000
4,Dallas Turner,DE,74.5,223.0,4.991,7.7520,4.4870,26.000,115.0000,83.5000,34.250000,9.840000,38.378617


##  Track & Field Data. Need to do some data cleaning (add ZCruit ID, drop duplicates, drop players with no T&F data) before running the master_fillna function.

Load in original uncleaned ZCruit Data as `original_data` from `original_zcruit_data.csv`. Select relevant columns.

In [25]:
original_data = pd.read_csv("original_zcruit_data.csv", error_bad_lines=False)
good_columns = ["Zcruit ID", "Full Name", "Grad Year", "Position", "Height Source", "Weight Source", "Committed To"]
original_data = original_data[good_columns]

b'Skipping line 1845: expected 70 fields, saw 72\nSkipping line 3345: expected 70 fields, saw 72\n'
b'Skipping line 11921: expected 70 fields, saw 72\nSkipping line 12609: expected 70 fields, saw 72\nSkipping line 13420: expected 70 fields, saw 72\nSkipping line 14108: expected 70 fields, saw 72\nSkipping line 14512: expected 70 fields, saw 72\n'
b'Skipping line 16512: expected 70 fields, saw 72\nSkipping line 21242: expected 70 fields, saw 72\nSkipping line 23743: expected 70 fields, saw 72\n'
b'Skipping line 58041: expected 70 fields, saw 72\nSkipping line 60541: expected 70 fields, saw 72\n'
b'Skipping line 70025: expected 70 fields, saw 72\nSkipping line 72525: expected 70 fields, saw 72\n'
C:\Users\AlvinYu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4,8,40,48,57,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Merge uncleaned ZCruit data, `original_data`, with cleaned ZCruit data, `data` to add ZCruit ID to `data`. Save merged dataset as `data_with_zcruitid`. Show first 5 rows. Scroll all the way to the right to see `Zcruit ID` column - we didn't have this on our cleaned dataset so we have to add it back in.

In [26]:
data_with_zcruitid = data.merge(original_data, how = "left", on = ["Full Name", "Grad Year", "Position", "Height Source", "Weight Source", "Committed To"])
data_with_zcruitid = data_with_zcruitid.drop_duplicates()
data_with_zcruitid.head(5)

,Full Name,Last Name,Grad Year,Position,Height,Height Source,Weight,Weight Source,Zcruit Rating,Committed To,...,Powerball Toss,Powerball Toss Source,SPARQ Rating,SPARQ Rating Source,GPA,ACT,SAT,Unnamed: 37,Unnamed: 38,Zcruit ID
0,Korey Foreman,Foreman,2021,DE,76.0,nike_opening,248.0,nike_opening,96.6970,USC,...,44.5,nike_opening,98.76,nike_opening,3.03,NaN,NaN,NaN,NaN,2481c4e2-f105-4e9f-b85d-0925a343910b
2,J.T. Tuimoloau,Tuimoloau,2021,DE,76.0,national_preps,275.0,national_preps,96.6970,NaN,...,NaN,NaN,NaN,NaN,3.00,NaN,NaN,NaN,NaN,c4fcba47-0337-4047-bc4d-0187da6f698e
4,JC Latham,Latham,2021,OT,78.0,national_preps,279.0,nike_opening,96.6970,Alabama,...,NaN,NaN,NaN,NaN,2.35,NaN,NaN,NaN,NaN,1db04d99-cca6-4bfc-8a5a-3d40ae1795e0
6,Emeka Egbuka,Egbuka,2021,WR,72.5,nike_opening,190.0,nike_opening,96.3939,Ohio State,...,41.0,nike_opening,NaN,NaN,4.00,NaN,NaN,NaN,NaN,abda246f-bb45-4b10-baf1-a9e5d508cc84
8,Dallas Turner,Turner,2021,DE,74.5,rivals_camp,223.0,rivals_camp,96.3939,Alabama,...,NaN,NaN,NaN,NaN,3.70,NaN,NaN,NaN,NaN,7351f453-1854-48ba-aa2c-e6eb9f4b9859


Load in cleaned track & field data as `tf_data` from `cleaned_tf_data.csv`. The cleaning process conducted to generate `cleaned_tf_data.csv` can be seen in the `Track and Field Cleaning [FINAL]` notebook.

In [27]:
tf_data = pd.read_csv("cleaned_tf_data.csv", sep=",")
tf_data = tf_data.drop(columns = 'Unnamed: 0')
tf_data.head(5)

,Zcruit ID,Player Name,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95,NaN,NaN,66.0,217.0,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90,NaN,NaN,NaN,NaN,NaN,490.5,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51,24.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74,25.77,NaN,47.84,NaN,207.5,NaN,NaN,NaN


Merge dataset with Zcruit ID, `data_with_zcruitid`, with track and field data, `tf_data` to create `tf_data_with_pos`, the dataset ready to be cleaned. `tf_data_with_pos` should be a cleaned track and field dataset with an added Position column to be used by the master cleaning function, `master_fillna`.

In [28]:
merge_with_tf = data_with_zcruitid[["Zcruit ID", "Position"]]
tf_data_with_pos = tf_data.merge(merge_with_tf, how = "left", on = "Zcruit ID")
tf_data_with_pos = tf_data_with_pos.rename(columns = {'Player Name': 'Full Name'})
tf_data_with_pos.head(5)

,Zcruit ID,Full Name,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus,Position
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95,NaN,NaN,66.0,217.0,NaN,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90,NaN,NaN,NaN,NaN,NaN,490.5,NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51,24.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CB
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74,25.77,NaN,47.84,NaN,207.5,NaN,NaN,NaN,RB


Drop position-filtered athletes with NO track or field metrics. Threshold is 4 because a player with no track and field metrics should only have 3 values (ID, Name, Pos) and nothing else if completely empty. Continue to save data as `tf_data_with_pos`.

In [29]:
tf_data_with_pos = tf_data_with_pos.dropna(thresh = 4)
tf_data_with_pos.head(5)

,Zcruit ID,Full Name,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus,Position
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95,NaN,NaN,66.0,217.0,NaN,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90,NaN,NaN,NaN,NaN,NaN,490.5,NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51,24.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CB
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74,25.77,NaN,47.84,NaN,207.5,NaN,NaN,NaN,RB
5,c98e0321-f3a4-4b85-ab2a-bbd16d46ea86,Isaac Jones,11.73,23.28,NaN,NaN,NaN,194.5,NaN,NaN,NaN,RB


### Option 1: Fill in Track and Field Data Yourself 

Code below iterates through all the positions and creates a dictionary called `cleaned_tf_metrics_position_dataframes`  where the keys are the positions and the values are the cleaned datasets for each position using all of the track and field events (`track_events` and `field_events`) and the `tf_data_with_pos` dataset. 

In [30]:
%%time
cleaned_tf_metrics_position_dataframes = {}
for position in positions:
    cleaned_tf_metrics_position_dataframes[position] = master_fillna(tf_data_with_pos, position, track_events + field_events)

WR
{'100 Meter': 'There are 470 NaNs out of 1474 players', '200 Meter': 'There are 403 NaNs out of 1474 players', '110 Meter Hurdles': 'There are 1312 NaNs out of 1474 players', '300 Meter Hurdles': 'There are 1311 NaNs out of 1474 players', 'High Jump': 'There are 1167 NaNs out of 1474 players', 'Long Jump': 'There are 768 NaNs out of 1474 players', 'Triple Jump': 'There are 1177 NaNs out of 1474 players', 'Shotput': 'There are 1427 NaNs out of 1474 players', 'Discus': 'There are 1444 NaNs out of 1474 players'}
100 Meter ['200 Meter' '300 Meter Hurdles']
[0.31758863] 4.219561642764204
The RMSE of the model on the training data is 0.5621377260998385. The average 100 Meter of the training data is 12.094736842105252. On average, predictions are 0.04647787987770644 from the actual values in the training data
200 Meter ['100 Meter' '110 Meter Hurdles']


<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_metric_with_predictor_nulls[metric] = metric_mean
<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_metric_with_predictor_nulls[metric] = metric_mean
<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

[1.5397188] 6.1742857289888775
The RMSE of the model on the training data is 1.2377443899706517. The average 200 Meter of the training data is 24.79677944862156. On average, predictions are 0.04991553006047555 from the actual values in the training data
110 Meter Hurdles ['300 Meter Hurdles' 'Discus' '200 Meter']
[0.31671364] 3.77111980438878
The RMSE of the model on the training data is 1.4796834930099672. The average 110 Meter Hurdles of the training data is 18.078558558558555. On average, predictions are 0.08184742650898301 from the actual values in the training data
300 Meter Hurdles ['Discus' '110 Meter Hurdles' '100 Meter']
[0.0299289] 13.402207489960972
The RMSE of the model on the training data is 1.551026895201722. The average 300 Meter Hurdles of the training data is 44.14666666666667. On average, predictions are 0.03513349958928697 from the actual values in the training data
High Jump ['Triple Jump' 'Long Jump']
[0.06761343] 33.369442824787136
The RMSE of the model on the tr

<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Discus ['300 Meter Hurdles' '110 Meter Hurdles']
[23.80155095] -23.509135753591863
The RMSE of the model on the training data is 43.73978091057082. The average Discus of the training data is 1027.25. On average, predictions are 0.042579489813162154 from the actual values in the training data
100 Meter
There are 470 NaN values for this metric
There are 0 NaN values for this metric
200 Meter
There are 403 NaN values for this metric
There are 0 NaN values for this metric
110 Meter Hurdles
There are 1312 NaN values for this metric
There are 0 NaN values for this metric
300 Meter Hurdles
There are 1311 NaN values for this metric
There are 0 NaN values for this metric
High Jump
There are 1167 NaN values for this metric
There are 0 NaN values for this metric
Long Jump
There are 768 NaN values for this metric
There are 0 NaN values for this metric
Triple Jump
There are 1177 NaN values for this metric
There are 0 NaN values for this metric
Shotput
There are 1427 NaN values for this metric
There

...

<ipython-input-17-da4197b6c9a0>:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_metric_with_predictor_nulls[metric] = metric_mean
<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

[0.2922722] 5.2160218206908535
The RMSE of the model on the training data is 0.3908311892622584. The average 100 Meter of the training data is 13.25. On average, predictions are 0.02949669352922705 from the actual values in the training data
200 Meter ['100 Meter']
[1.96471179] 1.4555688482550941
The RMSE of the model on the training data is 1.0133159680961183. The average 200 Meter of the training data is 27.488. On average, predictions are 0.03686393946799034 from the actual values in the training data
110 Meter Hurdles []
300 Meter Hurdles []
High Jump []
Long Jump []
Triple Jump []
Shotput ['Discus']
[0.11964887] 291.18577404971495
The RMSE of the model on the training data is 45.91363572953311. The average Shotput of the training data is 417.13461538461536. On average, predictions are 0.11006910967386115 from the actual values in the training data
Discus ['Shotput']
[2.76699012] -101.55351395098478
The RMSE of the model on the training data is 220.79599481856448. The average Discu

<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

Discus ['Shotput']
[2.52123532] 94.47865562029347
The RMSE of the model on the training data is 183.95348440201985. The average Discus of the training data is 1162.6. On average, predictions are 0.15822594564082218 from the actual values in the training data
100 Meter
There are 12 NaN values for this metric
There are 0 NaN values for this metric
200 Meter
There are 11 NaN values for this metric
There are 0 NaN values for this metric
110 Meter Hurdles
There are 13 NaN values for this metric
There are 13 NaN values for this metric
300 Meter Hurdles
There are 13 NaN values for this metric
There are 13 NaN values for this metric
High Jump
There are 13 NaN values for this metric
There are 13 NaN values for this metric
Long Jump
There are 12 NaN values for this metric
There are 0 NaN values for this metric
Triple Jump
There are 11 NaN values for this metric
There are 0 NaN values for this metric
Shotput
There are 2 NaN values for this metric
There are 0 NaN values for this metric
Discus
Ther

<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles[metric + " " + "Percentile"] = only_percentiles.mean(axis=1)
<ipython-input-17-da4197b6c9a0>:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_percentiles['Predicted' + " " + metric] = only_percentiles.apply(lambda x: percentile_to_value(metric, x[metric + " " + "Percentile"], metric_dict), axis = 1)
<ipython-input-17-da4197b6c9a0>:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

An example of accessing the cleaned, filled-in T&F data for the WR position using the `cleaned_tf_metrics_position_dataframes` dictionary. Show first 5 rows.

In [31]:
cleaned_tf_metrics_position_dataframes['WR'].head(5)

,Zcruit ID,Full Name,Position,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
6,7e546367-e3bd-44b4-a1e7-1b2cf182d224,David Godsey,WR,11.170000,23.18,17.91,44.81,64.0,222.75,461.5000,471.597175,982.0
9,546e641d-b263-40a2-bcf4-7eea98eed7c7,Nyckoles Harbor,WR,11.311316,22.33,17.91,44.81,64.0,222.75,461.5000,508.917400,982.0
16,7f671d52-520b-45ab-93d3-4eb3b59ab5eb,Armond Scott,WR,11.667015,23.45,17.91,44.81,64.0,222.75,461.5000,460.904250,982.0
17,1bb159dd-b94a-4684-8493-fde202510adc,Jordan Doucet,WR,11.240000,24.31,17.91,44.81,64.0,276.75,517.4765,460.908300,982.0
18,40efd2c2-389a-46d1-af39-065243eeb086,Markevus Jackson,WR,11.710000,22.88,17.91,44.81,64.0,222.75,461.5000,455.225625,982.0


#### Combine all filled-in track and field datasets by position into one master filled-in track and field dataset called df_tf.
 Master combined cleaned and filled in track and field dataset is saved as `df_tf` variable. 

In [32]:
column_names = cleaned_tf_metrics_position_dataframes['QB'].columns
df_tf = pd.DataFrame(columns = column_names)
for position in positions:
    df_tf = df_tf.append(cleaned_tf_metrics_position_dataframes[position])
df_tf = df_tf.sort_index()

In [33]:
df_tf.head(5)

,Zcruit ID,Full Name,Position,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,CB,12.51,24.94,18.455,45.845,66.0,212.50,450.875000,378.750000,1103.0
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,RB,11.74,25.77,17.520,47.840,63.0,207.50,438.017729,436.750000,831.0
5,c98e0321-f3a4-4b85-ab2a-bbd16d46ea86,Isaac Jones,RB,11.73,23.28,17.520,44.910,63.0,194.50,423.156296,436.750000,831.0
6,7e546367-e3bd-44b4-a1e7-1b2cf182d224,David Godsey,WR,11.17,23.18,17.910,44.810,64.0,222.75,461.500000,471.597175,982.0
7,20396bae-d68b-4b18-858e-e341d20f2b85,Hunt Young,QB,12.22,24.68,17.650,45.240,64.0,217.75,451.750000,430.250000,1173.5


Like the ZCruit data, some metrics remain completely empty because there are no players at that position with a value for that event. The following code shows which events remain completely empty for each position. For example, at OL, there are no players who have run the 110 Meter Hurdles or done the Triple Jump (which makes sense). The dictionary of metrics and empty values is called `still_nan_events_for_positions_tf`

In [34]:
still_nan_events_for_positions_tf = {}
for position in positions:
    series = df_tf[df_tf['Position'] == position].isna().sum() > 0
    still_nan_events_for_positions_tf[position] = series[series].index.values
still_nan_events_for_positions_tf

{'WR': array([], dtype=object),
 'RB': array([], dtype=object),
 'LB': array([], dtype=object),
 'QB': array([], dtype=object),
 'DE': array([], dtype=object),
 'CB': array([], dtype=object),
 'OL': array(['110 Meter Hurdles', 'Triple Jump'], dtype=object),
 'DT': array(['High Jump'], dtype=object),
 'S': array([], dtype=object),
 'OT': array(['110 Meter Hurdles', '300 Meter Hurdles', 'Long Jump',
        'Triple Jump'], dtype=object),
 'OG': array(['110 Meter Hurdles', '300 Meter Hurdles', 'High Jump', 'Long Jump',
        'Triple Jump'], dtype=object),
 'TE': array([], dtype=object),
 'ATH': array([], dtype=object),
 'DB': array([], dtype=object),
 'DL': array([], dtype=object),
 'OC': array(['300 Meter Hurdles', 'High Jump', 'Long Jump', 'Triple Jump'],
       dtype=object),
 'K': array(['300 Meter Hurdles'], dtype=object),
 'FB': array(['300 Meter Hurdles'], dtype=object),
 'LS': array(['110 Meter Hurdles', '300 Meter Hurdles', 'High Jump'],
       dtype=object),
 'P': array(['110 

### Option 2: Load in Filled In Track & Field Data.
Read the master cleaned and filled in track and field dataset called `cleaned_filled_tf_data.csv`. This dataset is the result of all the work in Option 1. Do this if you don't want to run the cells in Option 1 which can take up to a minute. Save dataset as variable called `filled_tf`. Dataset should look exactly like `df_tf`

In [35]:
filled_tf = pd.read_csv('cleaned_filled_tf_data.csv')

In [36]:
filled_tf = filled_tf.drop(columns = 'Unnamed: 0')
filled_tf.head(5)

,Zcruit ID,Full Name,Position,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
0,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,CB,12.51,24.94,18.455,45.845,66.0,212.50,450.875000,378.750000,1103.0
1,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,RB,11.74,25.77,17.520,47.840,63.0,207.50,438.017729,436.750000,831.0
2,c98e0321-f3a4-4b85-ab2a-bbd16d46ea86,Isaac Jones,RB,11.73,23.28,17.520,44.910,63.0,194.50,423.156296,436.750000,831.0
3,7e546367-e3bd-44b4-a1e7-1b2cf182d224,David Godsey,WR,11.17,23.18,17.910,44.810,64.0,222.75,461.500000,471.597175,982.0
4,20396bae-d68b-4b18-858e-e341d20f2b85,Hunt Young,QB,12.22,24.68,17.650,45.240,64.0,217.75,451.750000,430.250000,1173.5
